# Image Quality

The objective is to quantify the quality of filtered images with the following metrics, with $M$ as the original image, $F$ as the filtered image, $\mu$ as mean, and $\sigma$ as the standard deviation:
1. Speckle Suppression Index:	$SSI = \frac{s_F}{\mu_F} \cdot \frac{\mu_M}{s_M}$ [1]
2. Speckle Suppresion and Mean Preservation Index: $SMPI = (1 + |\mu_M - \mu_F|) \cdot \frac{\sigma_F}{\sigma_M}$	[1]
3. Mean Preservation Index: $MPI = |\frac{\mu_M - \mu_F}{\mu_M}|$ [1]
4. Mean Preservation Speckle Suppression Index: $MPSSI = |1 - \frac{\mu_F}{\mu_M}| \cdot \frac{\sigma_F}{\sigma_M}=MPI \cdot \frac{\sigma_F}{\sigma_M}$ [1]
5. Equivalent Number of Looks: $ENL=\frac{\mu_F^2}{\sigma_F^2}$ [1]
6. Edge Save Index Horizontal/Vertical [2, 3]. Let $P$ be the maximum row number and $Q$ be the maximum column.
- $ESIH=\frac{\sum_{m=1}^{P} \sum_{n=1}^{Q-1} |F_{m,n+1} - F{m,n}|}{\sum_{m=1}^{P}\sum_{n=1}^{Q-1} |M_{m,n+1} - M{m,n}|}$ 
- $ESIV=\frac{\sum_{n=1}^{Q} \sum_{m=1}^{P-1} |F_{m+1,n} - F{m,n}|}{\sum_{n=1}^{Q}\sum_{m=1}^{P-1} |M_{m+1,n} - M{m,n}|}$ 

References:
1. Dellepiane, S. G., & Angiati, E. (2014). Quality Assessment of Despeckled SAR Images. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 7(2), 691–707. https://doi.org/10.1109/JSTARS.2013.2279501
2. Shitole, S., Jain, V., & Vanama, V. S. K. (2020). De-Speckling of Synthetic Aperture Radar Using Discrete Fourier Transform. IGARSS 2020 - 2020 IEEE International Geoscience and Remote Sensing Symposium, 1524–1527. https://doi.org/10.1109/IGARSS39084.2020.9324277
3. Zhang, W., Liu, F., Jiao, L., Hou, B., Wang, S., & Shang, R. (2010). SAR Image Despeckling Using Edge Detection and Feature Clustering in Bandelet Domain. IEEE Geoscience and Remote Sensing Letters, 7(1), 131–135. https://doi.org/10.1109/LGRS.2009.2028588


In [3]:
import pandas as pd
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
def ssi(original, filtered): 
    # Speckle Suppression Index (SSI) (smaller, better)
    return (np.std(filtered) / np.mean(filtered)) * (np.mean(original) / np.std(original))

def smpi(original, filtered):
    # Speckle Suppression and Mean Preservation Index (SMPI) (smaller, better)
    Q = 1 + np.abs(np.mean(original) - np.mean(filtered))
    return Q * np.std(filtered) / np.std(original)

def mpi(original, filtered):
    # Mean Preservation Index (MPI), (smaller, better)
    return np.abs((np.mean(original) - np.mean(filtered)) / np.mean(original))
    
def mpssi(original, filtered):
    # Mean Preservation Speckle Suppression Index (MPSSI), (smaller, better)
    return mpi(original, filtered) * (np.std(filtered) / np.std(original))
    
def enl(image):
    # Equivalent Number of Looks (ENL) (higher, better)
    return (np.mean(image) ** 2) / np.var(image)

# def esih(original, filtered):
#     # Edge Save Index Horizontal (ESIH)
#     rows, cols = original.shape
#     numerator = 0
#     denominator = 0 
#     for row in range(rows):
#         for col in range(cols - 1):
#             numerator += np.abs(filtered[row, col+1] - filtered[row, col])
#             denominator += np.abs(original[row, col+1] - original[row, col])
#     return numerator / denominator

def esih(original, filtered):
    # Efficient Implementation of Edge Save Index Horizontal (ESIH)
    rows, cols = original.shape
    numerator = np.sum(np.abs(filtered[:, 1:cols] - filtered[:, :cols-1]).flatten())
    denominator = np.sum(np.abs(original[:, 1:cols] - original[:, :cols-1]).flatten())
    return numerator / denominator

# def esiv(original, filtered):
#     # Edge Save Index Vertical (ESIH)
#     rows, cols = original.shape 
#     numerator = 0 
#     denominator = 0 
#     for col in range(cols):
#         for row in range(rows - 1):
#             numerator += np.abs(filtered[row+1, col] - filtered[row, col])
#             denominator += np.abs(original[row+1, col] - original[row, col])
#     return numerator / denominator

def esiv(original, filtered):
    # Efficient Implementation of Edge Save Index Vertical (ESIH)
    rows, cols = original.shape 
    numerator = np.sum(np.abs(filtered[1:rows, :] - filtered[:rows-1, :]).flatten())
    denominator = np.sum(np.abs(original[1:rows, :] - original[:rows-1, :]).flatten())
    return numerator / denominator

## Demo

In [75]:
original = cv2.imread('../results/selected_images/sentinel-1_image_VH_2018-08-12.png', cv2.IMREAD_GRAYSCALE)
boxcar = cv2.imread('../results/boxcar/sentinel-1_image_VH_2018-08-12.png', cv2.IMREAD_GRAYSCALE)
bm3d = cv2.imread('../results/bm3d_images/sigma_psd_20/sentinel-1_image_VH_2018-08-12.png', cv2.IMREAD_GRAYSCALE)

original = original.astype('int')
boxcar = boxcar.astype('int')
bm3d = bm3d.astype('int')

In [79]:
print('Boxcar vs BM3d')
print(f'SSI: {ssi(original, boxcar)}, {ssi(original, bm3d)}')
print(f'SMPI: {smpi(original, boxcar)}, {smpi(original, bm3d)}')
print(f'MPI: {mpi(original, boxcar)}, {mpi(original, bm3d)}')
print(f'MPSSI: {mpssi(original, boxcar)}, {mpssi(original, bm3d)}')
print(f'ENL: {enl(boxcar)}, {enl(bm3d)}; original={enl(original)}')
print(f'ESIH: {esih(original, boxcar)}, {esih(original, bm3d)}')
print(f'ESIV: {esiv(original, boxcar)}, {esiv(original, bm3d)}')

Boxcar vs BM3d
SSI: 0.8527409471508081, 0.8472197217110418
SMPI: 0.9831994657748272, 0.8540640112075921
MPI: 0.0010436035700360858, 5.430673222531976e-05
MPSSI: 0.0008889947694242559, 4.601223320130614e-05
ENL: 101.96395310703274, 103.29725625598454; original=74.1448344252108
ESIH: 0.33061854026762366, 0.2718946134860498
ESIV: 0.32468299004430756, 0.25180353413458806


## Batch Processing

### Classical Methods

In [82]:
directories = {
    'Unfiltered': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/selected_images',
    'Mean Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/boxcar',
    'Median Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/median',
    'Lee Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/lee',
    'Frost Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/frost',
    'Gamma MAP Filter': '/Users/jimuelcelestejr/Documents/codebook/sar-speckle-noise-filtering/results/gamma_map',
}

files = os.listdir(directories['Unfiltered'])
files

['sentinel-1_image_VH_2018-08-12.png',
 'sentinel-1_image_VH_2018-12-10.png',
 'sentinel-1_image_VH_2018-10-11.png',
 'sentinel-1_image_VH_2019-04-09.png',
 'sentinel-1_image_VH_2019-03-04.png',
 'sentinel-1_image_VH_2018-09-05.png',
 'sentinel-1_image_VH_2019-01-03.png',
 'sentinel-1_image_VH_2019-02-08.png',
 'sentinel-1_image_VH_2018-11-04.png']

In [83]:
metrics = {
    'SSI': ssi, 
    'SMPI': smpi, 
    'MPI': mpi,
    'MPSSI': mpssi,
    'ENL': enl,
    'ESIH': esih,
    'ESIV': esiv
}
metrics

{'SSI': <function __main__.ssi(original, filtered)>,
 'SMPI': <function __main__.smpi(original, filtered)>,
 'MPI': <function __main__.mpi(original, filtered)>,
 'MPSSI': <function __main__.mpssi(original, filtered)>,
 'ENL': <function __main__.enl(image)>,
 'ESIH': <function __main__.esih(original, filtered)>,
 'ESIV': <function __main__.esiv(original, filtered)>}

In [96]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in files:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


Mean Filter SSI sentinel-1_image_VH_2018-08-12.png
Mean Filter SSI sentinel-1_image_VH_2018-12-10.png
Mean Filter SSI sentinel-1_image_VH_2018-10-11.png
Mean Filter SSI sentinel-1_image_VH_2019-04-09.png
Mean Filter SSI sentinel-1_image_VH_2019-03-04.png
Mean Filter SSI sentinel-1_image_VH_2018-09-05.png
Mean Filter SSI sentinel-1_image_VH_2019-01-03.png
Mean Filter SSI sentinel-1_image_VH_2019-02-08.png
Mean Filter SSI sentinel-1_image_VH_2018-11-04.png
Mean Filter SMPI sentinel-1_image_VH_2018-08-12.png
Mean Filter SMPI sentinel-1_image_VH_2018-12-10.png
Mean Filter SMPI sentinel-1_image_VH_2018-10-11.png
Mean Filter SMPI sentinel-1_image_VH_2019-04-09.png
Mean Filter SMPI sentinel-1_image_VH_2019-03-04.png
Mean Filter SMPI sentinel-1_image_VH_2018-09-05.png
Mean Filter SMPI sentinel-1_image_VH_2019-01-03.png
Mean Filter SMPI sentinel-1_image_VH_2019-02-08.png
Mean Filter SMPI sentinel-1_image_VH_2018-11-04.png
Mean Filter MPI sentinel-1_image_VH_2018-08-12.png
Mean Filter MPI sentin

Lee Filter MPSSI sentinel-1_image_VH_2019-03-04.png
Lee Filter MPSSI sentinel-1_image_VH_2018-09-05.png
Lee Filter MPSSI sentinel-1_image_VH_2019-01-03.png
Lee Filter MPSSI sentinel-1_image_VH_2019-02-08.png
Lee Filter MPSSI sentinel-1_image_VH_2018-11-04.png
Lee Filter ENL sentinel-1_image_VH_2018-08-12.png
Lee Filter ENL sentinel-1_image_VH_2018-12-10.png
Lee Filter ENL sentinel-1_image_VH_2018-10-11.png
Lee Filter ENL sentinel-1_image_VH_2019-04-09.png
Lee Filter ENL sentinel-1_image_VH_2019-03-04.png
Lee Filter ENL sentinel-1_image_VH_2018-09-05.png
Lee Filter ENL sentinel-1_image_VH_2019-01-03.png
Lee Filter ENL sentinel-1_image_VH_2019-02-08.png
Lee Filter ENL sentinel-1_image_VH_2018-11-04.png
Lee Filter ESIH sentinel-1_image_VH_2018-08-12.png
Lee Filter ESIH sentinel-1_image_VH_2018-12-10.png
Lee Filter ESIH sentinel-1_image_VH_2018-10-11.png
Lee Filter ESIH sentinel-1_image_VH_2019-04-09.png
Lee Filter ESIH sentinel-1_image_VH_2019-03-04.png
Lee Filter ESIH sentinel-1_image_VH

Gamma MAP Filter ESIV sentinel-1_image_VH_2019-03-04.png
Gamma MAP Filter ESIV sentinel-1_image_VH_2018-09-05.png
Gamma MAP Filter ESIV sentinel-1_image_VH_2019-01-03.png
Gamma MAP Filter ESIV sentinel-1_image_VH_2019-02-08.png
Gamma MAP Filter ESIV sentinel-1_image_VH_2018-11-04.png


In [97]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
Mean Filter,0.80475,0.920858,0.00105658,0.000851102,129.091,0.309634,0.304016
Median Filter,0.78775,1.21173,0.0037399,0.00311364,136.044,0.320512,0.313105
Lee Filter,0.822673,0.824064,1.03651e-05,9.13883e-06,122.801,0.388927,0.380429
Frost Filter,0.825561,1.22625,0.0033016,0.0027994,122.334,0.389752,0.381315
Gamma MAP Filter,0.89124,28.2669,0.295478,0.185517,105.473,0.440015,0.447188


In [104]:
output.to_excel('../results/image_quality/classic_filters.xlsx')

### BM3D

In [113]:
directories = {f'BM3D_sigma_psd_{i}': f'../results/bm3d_images/sigma_psd_{i}' for i in [10, 20, 30, 40, 50]}
directories['Unfiltered'] = '../results/selected_images'
directories, files

({'BM3D_sigma_psd_10': '../results/bm3d_images/sigma_psd_10',
  'BM3D_sigma_psd_20': '../results/bm3d_images/sigma_psd_20',
  'BM3D_sigma_psd_30': '../results/bm3d_images/sigma_psd_30',
  'BM3D_sigma_psd_40': '../results/bm3d_images/sigma_psd_40',
  'BM3D_sigma_psd_50': '../results/bm3d_images/sigma_psd_50',
  'Unfiltered': '../results/selected_images'},
 ['sentinel-1_image_VH_2018-08-12.png',
  'sentinel-1_image_VH_2018-12-10.png',
  'sentinel-1_image_VH_2018-10-11.png',
  'sentinel-1_image_VH_2019-04-09.png',
  'sentinel-1_image_VH_2019-03-04.png',
  'sentinel-1_image_VH_2018-09-05.png',
  'sentinel-1_image_VH_2019-01-03.png',
  'sentinel-1_image_VH_2019-02-08.png',
  'sentinel-1_image_VH_2018-11-04.png'])

In [114]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in files:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_10 SSI sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_10 SMPI sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_10 SMPI 

BM3D_sigma_psd_30 SMPI sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_30 MPI sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_30 MPI sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_30 MPI sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_30 MPI sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_30 MPI sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_30 MPI sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_30 MPI sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_30 MPI sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_30 MPI sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_30 MPSSI sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_30 MPSSI sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_30 MPSSI sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_30 MPSSI sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_30 MPSSI sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_30 MPSSI sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_30 MPSSI sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_3

BM3D_sigma_psd_50 MPSSI sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_50 MPSSI sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_50 MPSSI sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_50 ENL sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_50 ENL sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_50 ENL sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_50 ENL sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_50 ENL sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_50 ENL sentinel-1_image_VH_2018-09-05.png
BM3D_sigma_psd_50 ENL sentinel-1_image_VH_2019-01-03.png
BM3D_sigma_psd_50 ENL sentinel-1_image_VH_2019-02-08.png
BM3D_sigma_psd_50 ENL sentinel-1_image_VH_2018-11-04.png
BM3D_sigma_psd_50 ESIH sentinel-1_image_VH_2018-08-12.png
BM3D_sigma_psd_50 ESIH sentinel-1_image_VH_2018-12-10.png
BM3D_sigma_psd_50 ESIH sentinel-1_image_VH_2018-10-11.png
BM3D_sigma_psd_50 ESIH sentinel-1_image_VH_2019-04-09.png
BM3D_sigma_psd_50 ESIH sentinel-1_image_VH_2019-03-04.png
BM3D_sigma_psd_50 ES

In [115]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
BM3D_sigma_psd_10,0.879903,0.886907,2.87737e-05,2.63748e-05,107.014,0.498588,0.481729
BM3D_sigma_psd_20,0.779564,0.796845,9.83627e-05,8.69745e-05,135.589,0.240964,0.220644
BM3D_sigma_psd_30,0.738726,0.739761,2.66901e-05,1.96717e-05,151.693,0.162213,0.146517
BM3D_sigma_psd_40,0.71879,0.729919,0.00010758,7.4268e-05,160.927,0.133886,0.121498
BM3D_sigma_psd_50,0.705872,0.730635,0.000201898,0.000129158,167.395,0.118742,0.108342


In [116]:
output.to_excel('../results/image_quality/bm3d_filter.xlsx')

### Boxcar + DFT Edge

In [117]:
directories = {f'radius_{i}': f'../results/boxcar_edge/radius_{i}' for i in range(1,10)}
directories['Unfiltered'] = '../results/selected_images'
directories, files

({'radius_1': '../results/boxcar_edge/radius_1',
  'radius_2': '../results/boxcar_edge/radius_2',
  'radius_3': '../results/boxcar_edge/radius_3',
  'radius_4': '../results/boxcar_edge/radius_4',
  'radius_5': '../results/boxcar_edge/radius_5',
  'radius_6': '../results/boxcar_edge/radius_6',
  'radius_7': '../results/boxcar_edge/radius_7',
  'radius_8': '../results/boxcar_edge/radius_8',
  'radius_9': '../results/boxcar_edge/radius_9',
  'Unfiltered': '../results/selected_images'},
 ['sentinel-1_image_VH_2018-08-12.png',
  'sentinel-1_image_VH_2018-12-10.png',
  'sentinel-1_image_VH_2018-10-11.png',
  'sentinel-1_image_VH_2019-04-09.png',
  'sentinel-1_image_VH_2019-03-04.png',
  'sentinel-1_image_VH_2018-09-05.png',
  'sentinel-1_image_VH_2019-01-03.png',
  'sentinel-1_image_VH_2019-02-08.png',
  'sentinel-1_image_VH_2018-11-04.png'])

In [118]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in files:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


radius_1 SSI sentinel-1_image_VH_2018-08-12.png
radius_1 SSI sentinel-1_image_VH_2018-12-10.png
radius_1 SSI sentinel-1_image_VH_2018-10-11.png
radius_1 SSI sentinel-1_image_VH_2019-04-09.png
radius_1 SSI sentinel-1_image_VH_2019-03-04.png
radius_1 SSI sentinel-1_image_VH_2018-09-05.png
radius_1 SSI sentinel-1_image_VH_2019-01-03.png
radius_1 SSI sentinel-1_image_VH_2019-02-08.png
radius_1 SSI sentinel-1_image_VH_2018-11-04.png
radius_1 SMPI sentinel-1_image_VH_2018-08-12.png
radius_1 SMPI sentinel-1_image_VH_2018-12-10.png
radius_1 SMPI sentinel-1_image_VH_2018-10-11.png
radius_1 SMPI sentinel-1_image_VH_2019-04-09.png
radius_1 SMPI sentinel-1_image_VH_2019-03-04.png
radius_1 SMPI sentinel-1_image_VH_2018-09-05.png
radius_1 SMPI sentinel-1_image_VH_2019-01-03.png
radius_1 SMPI sentinel-1_image_VH_2019-02-08.png
radius_1 SMPI sentinel-1_image_VH_2018-11-04.png
radius_1 MPI sentinel-1_image_VH_2018-08-12.png
radius_1 MPI sentinel-1_image_VH_2018-12-10.png
radius_1 MPI sentinel-1_image_V

radius_3 ESIH sentinel-1_image_VH_2018-08-12.png
radius_3 ESIH sentinel-1_image_VH_2018-12-10.png
radius_3 ESIH sentinel-1_image_VH_2018-10-11.png
radius_3 ESIH sentinel-1_image_VH_2019-04-09.png
radius_3 ESIH sentinel-1_image_VH_2019-03-04.png
radius_3 ESIH sentinel-1_image_VH_2018-09-05.png
radius_3 ESIH sentinel-1_image_VH_2019-01-03.png
radius_3 ESIH sentinel-1_image_VH_2019-02-08.png
radius_3 ESIH sentinel-1_image_VH_2018-11-04.png
radius_3 ESIV sentinel-1_image_VH_2018-08-12.png
radius_3 ESIV sentinel-1_image_VH_2018-12-10.png
radius_3 ESIV sentinel-1_image_VH_2018-10-11.png
radius_3 ESIV sentinel-1_image_VH_2019-04-09.png
radius_3 ESIV sentinel-1_image_VH_2019-03-04.png
radius_3 ESIV sentinel-1_image_VH_2018-09-05.png
radius_3 ESIV sentinel-1_image_VH_2019-01-03.png
radius_3 ESIV sentinel-1_image_VH_2019-02-08.png
radius_3 ESIV sentinel-1_image_VH_2018-11-04.png
radius_4 SSI sentinel-1_image_VH_2018-08-12.png
radius_4 SSI sentinel-1_image_VH_2018-12-10.png
radius_4 SSI sentinel-

radius_6 MPI sentinel-1_image_VH_2019-02-08.png
radius_6 MPI sentinel-1_image_VH_2018-11-04.png
radius_6 MPSSI sentinel-1_image_VH_2018-08-12.png
radius_6 MPSSI sentinel-1_image_VH_2018-12-10.png
radius_6 MPSSI sentinel-1_image_VH_2018-10-11.png
radius_6 MPSSI sentinel-1_image_VH_2019-04-09.png
radius_6 MPSSI sentinel-1_image_VH_2019-03-04.png
radius_6 MPSSI sentinel-1_image_VH_2018-09-05.png
radius_6 MPSSI sentinel-1_image_VH_2019-01-03.png
radius_6 MPSSI sentinel-1_image_VH_2019-02-08.png
radius_6 MPSSI sentinel-1_image_VH_2018-11-04.png
radius_6 ENL sentinel-1_image_VH_2018-08-12.png
radius_6 ENL sentinel-1_image_VH_2018-12-10.png
radius_6 ENL sentinel-1_image_VH_2018-10-11.png
radius_6 ENL sentinel-1_image_VH_2019-04-09.png
radius_6 ENL sentinel-1_image_VH_2019-03-04.png
radius_6 ENL sentinel-1_image_VH_2018-09-05.png
radius_6 ENL sentinel-1_image_VH_2019-01-03.png
radius_6 ENL sentinel-1_image_VH_2019-02-08.png
radius_6 ENL sentinel-1_image_VH_2018-11-04.png
radius_6 ESIH sentinel

radius_9 SSI sentinel-1_image_VH_2018-09-05.png
radius_9 SSI sentinel-1_image_VH_2019-01-03.png
radius_9 SSI sentinel-1_image_VH_2019-02-08.png
radius_9 SSI sentinel-1_image_VH_2018-11-04.png
radius_9 SMPI sentinel-1_image_VH_2018-08-12.png
radius_9 SMPI sentinel-1_image_VH_2018-12-10.png
radius_9 SMPI sentinel-1_image_VH_2018-10-11.png
radius_9 SMPI sentinel-1_image_VH_2019-04-09.png
radius_9 SMPI sentinel-1_image_VH_2019-03-04.png
radius_9 SMPI sentinel-1_image_VH_2018-09-05.png
radius_9 SMPI sentinel-1_image_VH_2019-01-03.png
radius_9 SMPI sentinel-1_image_VH_2019-02-08.png
radius_9 SMPI sentinel-1_image_VH_2018-11-04.png
radius_9 MPI sentinel-1_image_VH_2018-08-12.png
radius_9 MPI sentinel-1_image_VH_2018-12-10.png
radius_9 MPI sentinel-1_image_VH_2018-10-11.png
radius_9 MPI sentinel-1_image_VH_2019-04-09.png
radius_9 MPI sentinel-1_image_VH_2019-03-04.png
radius_9 MPI sentinel-1_image_VH_2018-09-05.png
radius_9 MPI sentinel-1_image_VH_2019-01-03.png
radius_9 MPI sentinel-1_image_V

In [119]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
radius_1,4.93458,132.954,0.242845,0.940788,3.77086,4.15335,4.14973
radius_2,5.13949,142.739,0.247194,0.9734,3.69995,4.42724,4.43037
radius_3,5.09666,140.151,0.251032,0.965205,3.43906,4.43087,4.43557
radius_4,4.97053,133.305,0.247975,0.943081,3.5177,4.34516,4.35054
radius_5,5.09075,140.038,0.247912,0.949179,3.51946,4.4824,4.48803
radius_6,5.10032,144.595,0.248011,0.951217,3.53315,4.55381,4.55376
radius_7,5.00531,134.509,0.247176,0.935007,3.5037,4.45983,4.46658
radius_8,5.18805,145.501,0.250656,0.981024,3.44314,4.68047,4.68766
radius_9,5.12424,142.357,0.248491,0.955447,3.42406,4.64839,4.65726


In [120]:
output.to_excel('../results/image_quality/boxcar_dft_edge_filter.xlsx')

### BM3D + DFT Edge

In [121]:
directories = {f'radius_{i}': f'../results/bm3d_edge/radius_{i}' for i in range(1,10)}
directories['Unfiltered'] = '../results/selected_images'
directories, files

({'radius_1': '../results/bm3d_edge/radius_1',
  'radius_2': '../results/bm3d_edge/radius_2',
  'radius_3': '../results/bm3d_edge/radius_3',
  'radius_4': '../results/bm3d_edge/radius_4',
  'radius_5': '../results/bm3d_edge/radius_5',
  'radius_6': '../results/bm3d_edge/radius_6',
  'radius_7': '../results/bm3d_edge/radius_7',
  'radius_8': '../results/bm3d_edge/radius_8',
  'radius_9': '../results/bm3d_edge/radius_9',
  'Unfiltered': '../results/selected_images'},
 ['sentinel-1_image_VH_2018-08-12.png',
  'sentinel-1_image_VH_2018-12-10.png',
  'sentinel-1_image_VH_2018-10-11.png',
  'sentinel-1_image_VH_2019-04-09.png',
  'sentinel-1_image_VH_2019-03-04.png',
  'sentinel-1_image_VH_2018-09-05.png',
  'sentinel-1_image_VH_2019-01-03.png',
  'sentinel-1_image_VH_2019-02-08.png',
  'sentinel-1_image_VH_2018-11-04.png'])

In [122]:
output = pd.DataFrame(columns=metrics.keys(), index=[d for d in directories if d != 'Unfiltered'])
for directory in directories:
    if directory != 'Unfiltered':        
        for metric in metrics:
            results = []
            for file in files:
                print(directory, metric, file)
                if metric == 'ENL':
                    image = os.path.join(directories[directory], file)
                    image = cv2.imread(image, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](image))
                else:
                    original = os.path.join(directories['Unfiltered'], file)
                    original = cv2.imread(original, cv2.IMREAD_GRAYSCALE).astype('int')
                    filtered = os.path.join(directories[directory], file)
                    filtered = cv2.imread(filtered, cv2.IMREAD_GRAYSCALE).astype('int')
                    results.append(metrics[metric](original, filtered))
            median = np.median(results)
            output.loc[directory, metric] = median


radius_1 SSI sentinel-1_image_VH_2018-08-12.png
radius_1 SSI sentinel-1_image_VH_2018-12-10.png
radius_1 SSI sentinel-1_image_VH_2018-10-11.png
radius_1 SSI sentinel-1_image_VH_2019-04-09.png
radius_1 SSI sentinel-1_image_VH_2019-03-04.png
radius_1 SSI sentinel-1_image_VH_2018-09-05.png
radius_1 SSI sentinel-1_image_VH_2019-01-03.png
radius_1 SSI sentinel-1_image_VH_2019-02-08.png
radius_1 SSI sentinel-1_image_VH_2018-11-04.png
radius_1 SMPI sentinel-1_image_VH_2018-08-12.png
radius_1 SMPI sentinel-1_image_VH_2018-12-10.png
radius_1 SMPI sentinel-1_image_VH_2018-10-11.png
radius_1 SMPI sentinel-1_image_VH_2019-04-09.png
radius_1 SMPI sentinel-1_image_VH_2019-03-04.png
radius_1 SMPI sentinel-1_image_VH_2018-09-05.png
radius_1 SMPI sentinel-1_image_VH_2019-01-03.png
radius_1 SMPI sentinel-1_image_VH_2019-02-08.png
radius_1 SMPI sentinel-1_image_VH_2018-11-04.png
radius_1 MPI sentinel-1_image_VH_2018-08-12.png
radius_1 MPI sentinel-1_image_VH_2018-12-10.png
radius_1 MPI sentinel-1_image_V

radius_3 ENL sentinel-1_image_VH_2018-11-04.png
radius_3 ESIH sentinel-1_image_VH_2018-08-12.png
radius_3 ESIH sentinel-1_image_VH_2018-12-10.png
radius_3 ESIH sentinel-1_image_VH_2018-10-11.png
radius_3 ESIH sentinel-1_image_VH_2019-04-09.png
radius_3 ESIH sentinel-1_image_VH_2019-03-04.png
radius_3 ESIH sentinel-1_image_VH_2018-09-05.png
radius_3 ESIH sentinel-1_image_VH_2019-01-03.png
radius_3 ESIH sentinel-1_image_VH_2019-02-08.png
radius_3 ESIH sentinel-1_image_VH_2018-11-04.png
radius_3 ESIV sentinel-1_image_VH_2018-08-12.png
radius_3 ESIV sentinel-1_image_VH_2018-12-10.png
radius_3 ESIV sentinel-1_image_VH_2018-10-11.png
radius_3 ESIV sentinel-1_image_VH_2019-04-09.png
radius_3 ESIV sentinel-1_image_VH_2019-03-04.png
radius_3 ESIV sentinel-1_image_VH_2018-09-05.png
radius_3 ESIV sentinel-1_image_VH_2019-01-03.png
radius_3 ESIV sentinel-1_image_VH_2019-02-08.png
radius_3 ESIV sentinel-1_image_VH_2018-11-04.png
radius_4 SSI sentinel-1_image_VH_2018-08-12.png
radius_4 SSI sentinel-

radius_6 MPI sentinel-1_image_VH_2019-01-03.png
radius_6 MPI sentinel-1_image_VH_2019-02-08.png
radius_6 MPI sentinel-1_image_VH_2018-11-04.png
radius_6 MPSSI sentinel-1_image_VH_2018-08-12.png
radius_6 MPSSI sentinel-1_image_VH_2018-12-10.png
radius_6 MPSSI sentinel-1_image_VH_2018-10-11.png
radius_6 MPSSI sentinel-1_image_VH_2019-04-09.png
radius_6 MPSSI sentinel-1_image_VH_2019-03-04.png
radius_6 MPSSI sentinel-1_image_VH_2018-09-05.png
radius_6 MPSSI sentinel-1_image_VH_2019-01-03.png
radius_6 MPSSI sentinel-1_image_VH_2019-02-08.png
radius_6 MPSSI sentinel-1_image_VH_2018-11-04.png
radius_6 ENL sentinel-1_image_VH_2018-08-12.png
radius_6 ENL sentinel-1_image_VH_2018-12-10.png
radius_6 ENL sentinel-1_image_VH_2018-10-11.png
radius_6 ENL sentinel-1_image_VH_2019-04-09.png
radius_6 ENL sentinel-1_image_VH_2019-03-04.png
radius_6 ENL sentinel-1_image_VH_2018-09-05.png
radius_6 ENL sentinel-1_image_VH_2019-01-03.png
radius_6 ENL sentinel-1_image_VH_2019-02-08.png
radius_6 ENL sentinel-

radius_9 SSI sentinel-1_image_VH_2019-03-04.png
radius_9 SSI sentinel-1_image_VH_2018-09-05.png
radius_9 SSI sentinel-1_image_VH_2019-01-03.png
radius_9 SSI sentinel-1_image_VH_2019-02-08.png
radius_9 SSI sentinel-1_image_VH_2018-11-04.png
radius_9 SMPI sentinel-1_image_VH_2018-08-12.png
radius_9 SMPI sentinel-1_image_VH_2018-12-10.png
radius_9 SMPI sentinel-1_image_VH_2018-10-11.png
radius_9 SMPI sentinel-1_image_VH_2019-04-09.png
radius_9 SMPI sentinel-1_image_VH_2019-03-04.png
radius_9 SMPI sentinel-1_image_VH_2018-09-05.png
radius_9 SMPI sentinel-1_image_VH_2019-01-03.png
radius_9 SMPI sentinel-1_image_VH_2019-02-08.png
radius_9 SMPI sentinel-1_image_VH_2018-11-04.png
radius_9 MPI sentinel-1_image_VH_2018-08-12.png
radius_9 MPI sentinel-1_image_VH_2018-12-10.png
radius_9 MPI sentinel-1_image_VH_2018-10-11.png
radius_9 MPI sentinel-1_image_VH_2019-04-09.png
radius_9 MPI sentinel-1_image_VH_2019-03-04.png
radius_9 MPI sentinel-1_image_VH_2018-09-05.png
radius_9 MPI sentinel-1_image_V

In [123]:
output

,SSI,SMPI,MPI,MPSSI,ENL,ESIH,ESIV
radius_1,4.93094,132.636,0.241983,0.93847,3.77917,4.10755,4.09655
radius_2,5.13597,142.419,0.246318,0.96861,3.70602,4.38223,4.37801
radius_3,5.09309,139.819,0.249942,0.960512,3.44584,4.38526,4.38261
radius_4,4.96679,132.962,0.246602,0.940572,3.523,4.29832,4.29613
radius_5,5.08889,139.702,0.246286,0.944648,3.52514,4.43654,4.43474
radius_6,5.09834,144.261,0.246399,0.946693,3.54038,4.50531,4.49736
radius_7,5.00149,134.153,0.246003,0.930488,3.50945,4.41301,4.41188
radius_8,5.18583,145.157,0.249722,0.976616,3.45,4.63535,4.63497
radius_9,5.12187,142.002,0.247541,0.951036,3.42722,4.60273,4.60397


In [124]:
output.to_excel('../results/image_quality/bm3d_dft_edge_filter.xlsx')